In [ ]:
from langchain_core.chat_sessions

In [12]:
store = {}

In [13]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [14]:
SESSION_ID = "230521"
SESSION_ID

'230521'

In [ ]:
def add_to_memory(session_id, input=True, output=True, context=None):
    memory = get_session_history(session_id)
    if input:
        memory.

In [10]:
from ..chain.chains import classify_message_chain 

ImportError: attempted relative import with no known parent package

In [11]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(__file__)) + '/..')
from chain.chains import classify_message_chain

NameError: name '__file__' is not defined

In [2]:
import sys
import os


# 현재 노트북 파일의 디렉토리 경로를 기준으로 부모 디렉토리를 Python 경로에 추가
sys.path.append(os.path.abspath('..'))

In [3]:
from chain.chains import classify_message_chain

ImproperlyConfigured: Requested setting INSTALLED_APPS, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

In [4]:
# Django 설정 초기화
import django
django.setup()
from chain.chains import classify_message_chain

ImproperlyConfigured: Requested setting LOGGING_CONFIG, but settings are not configured. You must either define the environment variable DJANGO_SETTINGS_MODULE or call settings.configure() before accessing settings.

In [14]:
response = classify_message_chain.invoke(
    {"input": "주문 변경할게"}
)
response

NameError: name 'classify_message_chain' is not defined

---

복붙해 실험

In [25]:
store = {}
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [21]:
SESSION_ID = '230521'
SESSION_ID

'230521'

In [26]:
memory = get_session_history(SESSION_ID)
memory

InMemoryChatMessageHistory(messages=[])

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

message_type_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into one of the following two types:
            - Product inquiry, order history inquiry, order change history inquiry, order cancellation history inquiry: '문의'
            - Order request, order change request, order cancellation request: '요청'
            
            You need to review the messages in the Messages Placeholder from the latest to the oldest.

            Consider the previous AI responses and their classifications to understand the intent behind the current input. 
            Use this context to make an accurate classification. 
            If the latest AI response was classified as '요청', and the current input is related to an order, it is likely a '요청'.
            
            Additionally, if the input contains order details, it should be classified as '요청'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [8]:
classify_message_chain = message_type_prompt | model

In [16]:
input = "주문 변경할게"

In [17]:
memory.add_user_message(input)
memory.messages

[HumanMessage(content='주문 변경할게')]

In [35]:
bool("")

False

In [ ]:
memory.add_user_message

In [36]:
def add_to_memory(session_id, invoked_runnable, input="", context=None):
    memory = get_session_history(session_id)
    if input:
        memory.memory.add_user_message(input)
    output = invoked_runnable
    output_content = output.content
    if context:
        output = context + "\n" + output_content
    memory.add_ai_message(output_content)
    
    


In [27]:
response = classify_message_chain.invoke(
    {"input": input,
     "chat_history": memory.messages}
)
response

AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 181, 'total_tokens': 184}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0d8eaac8-58b5-4227-8ee5-bed3229120cc-0')

In [29]:
context = "사용자 입력에 대한 유형 분류"
response_with_context = context + '->' + response.content
response_with_context

'사용자 입력에 대한 유형 분류->요청'

In [30]:
memory.add_user_message(input)
memory.add_ai_message(response_with_context)
memory.messages

[HumanMessage(content='주문 변경할게'), AIMessage(content='사용자 입력에 대한 유형 분류->요청')]

In [37]:
invoked_classify_message_chain = classify_message_chain.invoke(
    {"input": input,
     "chat_history": memory.messages}
)

In [ ]:
add_to_memory(SESSION_ID, )

In [64]:
store = {}
memory = get_session_history(SESSION_ID)
memory

InMemoryChatMessageHistory(messages=[])

In [65]:
input = "주문 취소할게"
invoked_runnable = lambda: classify_message_chain.invoke({
    "input": input,
    "chat_history": memory.messages
})

In [66]:
context

'사용자 입력에 대한 유형 분류'

In [67]:
def add_to_memory(session_id, invoked_runnable, input="", context=None):
    memory = get_session_history(session_id)
    if input:
        memory.add_user_message(input)
    output = invoked_runnable()
    output_content = output.content
    if context:
        output_content = context + "\n" + output_content
    memory.add_ai_message(output_content)

In [68]:
add_to_memory(SESSION_ID, invoked_runnable, input, context)

In [69]:
store[SESSION_ID]

InMemoryChatMessageHistory(messages=[HumanMessage(content='주문 취소할게'), AIMessage(content='사용자 입력에 대한 유형 분류\n요청')])

In [ ]:
invoked_runnable = lambda: classify_message_chain.invoke({
    "input": input,
    "chat_history": memory.messages
})

In [ ]:
def create_invoked_runnable(chain):
    lambda : chain.invoke()

---

In [72]:
from typing import List, Optional
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    save_mode: Optional[str] = Field(default="both")  # "input", "output", "both"
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """조건에 따라 메시지를 저장"""
        if self.save_mode == "input":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            self.messages.extend(input_messages)
        elif self.save_mode == "output":
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            self.messages.extend(output_messages)
        elif self.save_mode == "both":
            self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

store = {}

def get_by_session_id(session_id: str, save_mode: str = "both") -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory(save_mode=save_mode)
    return store[session_id]


In [73]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = prompt | model

# 세션에 따라 다른 저장 모드를 설정
chain_with_history_input_only = RunnableWithMessageHistory(
    chain,
    get_session_history=lambda session_id: get_by_session_id(session_id, "input"),
    input_messages_key="question",
    history_messages_key="history",
)

chain_with_history_output_only = RunnableWithMessageHistory(
    chain,
    get_session_history=lambda session_id: get_by_session_id(session_id, "output"),
    input_messages_key="question",
    history_messages_key="history",
)

chain_with_history_both = RunnableWithMessageHistory(
    chain,
    get_session_history=lambda session_id: get_by_session_id(session_id, "both"),
    input_messages_key="question",
    history_messages_key="history",
)

# 입력 메시지만 저장
response = chain_with_history_input_only.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "input_only"}}
)
print(response)

# 출력 메시지만 저장
response = chain_with_history_output_only.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "output_only"}}
)
print(response)

# 입력 및 출력 메시지 모두 저장
response = chain_with_history_both.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"session_id": "both"}}
)
print(response)

# 저장된 메시지 확인
print(store)


Parent run cc96e3d1-9b6d-40ef-baa6-d568d80bc9b8 not found for run 5f1a6d67-8dbc-4c29-a20a-e253f49fc61a. Treating as a root run.


content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle. In simpler terms, it is a function that relates the angle of a triangle to the ratio of the length of the side adjacent to that angle and the length of the hypotenuse. It is commonly denoted as cos(θ), where θ is the angle.' response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 25, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-ecd7c8ab-0b24-4bc3-8644-b54a0326623c-0'


Parent run 9bf22ae1-d251-407a-9764-d91548c019d8 not found for run 05bdce8f-1c18-4f20-975d-2216a75e12b6. Treating as a root run.


content='Cosine is a trigonometric function that represents the ratio of the length of the adjacent side to the length of the hypotenuse in a right triangle. In simpler terms, it is a function that relates the angle of a right triangle to the ratio of two sides of the triangle. The cosine function is commonly denoted as cos(θ), where θ represents the angle in the triangle.' response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 25, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-bdb5ba70-26ae-4ef5-a085-50d7368e90de-0'


Parent run 9b625df0-1e59-49bf-bbbf-537820b5f096 not found for run 9b3ed8a5-81b1-4dad-98fd-1ff55c737f17. Treating as a root run.


content='Cosine is a trigonometric function that relates the angle of a right triangle to the ratio of the length of the side adjacent to the angle to the length of the hypotenuse. In simpler terms, the cosine of an angle in a right triangle is calculated as the length of the side adjacent to the angle divided by the length of the hypotenuse. It is commonly abbreviated as cos.' response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 25, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-5ea96bf3-0d3e-4a4c-83d6-2005f98a3687-0'
{'input_only': InMemoryHistory(messages=[HumanMessage(content='What does cosine mean?')], save_mode='input'), 'output_only': InMemoryHistory(messages=[AIMessage(content='Cosine is a trigonometric function that represents the ratio of the length of the adjacent side to the length of the hypotenuse in a right triangle. In simpler terms, it is a function that relates

In [76]:
print(len(store["output_only"].messages))
store["output_only"].messages

1


[AIMessage(content='Cosine is a trigonometric function that represents the ratio of the length of the adjacent side to the length of the hypotenuse in a right triangle. In simpler terms, it is a function that relates the angle of a right triangle to the ratio of two sides of the triangle. The cosine function is commonly denoted as cos(θ), where θ represents the angle in the triangle.', response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 25, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bdb5ba70-26ae-4ef5-a085-50d7368e90de-0')]

---

In [88]:
from typing import Dict, Any, Optional
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import Runnable
from langchain_community.chat_models import ChatAnthropic

# In-memory chat message history implementation
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    save_mode: Optional[str] = Field(default="both")  # "input", "output", "both"
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """조건에 따라 메시지를 저장"""
        if self.save_mode == "input":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            self.messages.extend(input_messages)
        elif self.save_mode == "output":
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            self.messages.extend(output_messages)
        elif self.save_mode == "both":
            self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

# Store for session histories
store = {}

def get_by_session_id(session_id: str, save_mode: str = "both") -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory(save_mode=save_mode)
    return store[session_id]

# RunnableWithMessageHistory class definition
class RunnableWithMessageHistory(Runnable):
    def __init__(self, runnable: Runnable, get_session_history, input_messages_key: str, history_messages_key: str, context_key: Optional[str] = None):
        self.runnable = runnable
        self.get_session_history = get_session_history
        self.input_messages_key = input_messages_key
        self.history_messages_key = history_messages_key
        self.context_key = context_key
    
    def invoke(self, input: Dict[str, Any], config: Optional[Dict[str, Any]] = None) -> Any:
        session_id = config["configurable"]["session_id"]
        history = self.get_session_history(session_id)
        
        current_input = input[self.input_messages_key]
        
        if isinstance(current_input, str):
            current_input_message = HumanMessage(content=current_input)
            history.add_messages([current_input_message])
        
        result = self.runnable.invoke(input, config)
        
        if isinstance(result, AIMessage):
            if self.context_key and self.context_key in input:
                context = input[self.context_key]
                result_with_context = AIMessage(content=f"{context}\n{result.content}")
                history.add_messages([result_with_context])
            else:
                history.add_messages([result])
        
        return result

# Example usage
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

# Using a simple fake model for demonstration purposes
chain = prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
    context_key="context"
)


In [86]:

# First interaction
result = chain_with_history.invoke(
    {"ability": "math", "question": "What does cosine mean?", "context": "Classifying user input"},
    config={"configurable": {"session_id": "session_1"}}
)

print(result)


KeyError: "Input to ChatPromptTemplate is missing variables {'history'}.  Expected: ['ability', 'history', 'question'] Received: ['ability', 'question', 'context']"

In [89]:
result = chain_with_history.invoke(
    {"ability": "math", "question": "What does cosine mean?", "context": "Classifying user input"},
    config={"configurable": {"session_id": "session_1"}}
)
result

KeyError: "Input to ChatPromptTemplate is missing variables {'history'}.  Expected: ['ability', 'history', 'question'] Received: ['ability', 'question', 'context']"

In [ ]:
result = chain_with_history.invoke(
    {"ability": "math", "question": "What does cosine mean?", "context": "Classifying user input"},
    config={"configurable": {"session_id": "session_1"}}
)

In [87]:

# Second interaction
result = chain_with_history.invoke(
    {"ability": "math", "question": "What's its inverse?", "context": "Classifying user input"},
    config={"configurable": {"session_id": "session_1"}}
)

print(result)


KeyError: "Input to ChatPromptTemplate is missing variables {'history'}.  Expected: ['ability', 'history', 'question'] Received: ['ability', 'question', 'context']"

In [ ]:

# Print the history to see the stored messages
history = get_by_session_id("session_1")
for message in history.messages:
    print(f"{message.__class__.__name__}: {message.content}")


---

In [90]:
from typing import List, Optional, Dict, Any
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, AIMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import Runnable
from langchain_community.chat_models import ChatAnthropic

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    save_mode: Optional[str] = Field(default="both")  # "input", "output", "both"
    
    def add_messages(self, messages: List[BaseMessage]) -> None:
        """조건에 따라 메시지를 저장"""
        if self.save_mode == "input":
            input_messages = [msg for msg in messages if isinstance(msg, HumanMessage)]
            self.messages.extend(input_messages)
        elif self.save_mode == "output":
            output_messages = [msg for msg in messages if isinstance(msg, AIMessage)]
            self.messages.extend(output_messages)
        elif self.save_mode == "both":
            self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

store = {}

def get_by_session_id(session_id: str, save_mode: str = "both") -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory(save_mode=save_mode)
    return store[session_id]

class RunnableWithMessageHistory(Runnable):
    def __init__(self, runnable: Runnable, get_session_history, input_messages_key: str, history_messages_key: str, context_key: Optional[str] = None):
        self.runnable = runnable
        self.get_session_history = get_session_history
        self.input_messages_key = input_messages_key
        self.history_messages_key = history_messages_key
        self.context_key = context_key
    
    def invoke(self, input: Dict[str, Any], config: Optional[Dict[str, Any]] = None) -> Any:
        session_id = config["configurable"]["session_id"]
        history = self.get_session_history(session_id)
        
        current_input = input[self.input_messages_key]
        
        if isinstance(current_input, str):
            current_input_message = HumanMessage(content=current_input)
            history.add_messages([current_input_message])
        
        input[self.history_messages_key] = history.messages
        
        result = self.runnable.invoke(input, config)
        
        if isinstance(result, AIMessage):
            if self.context_key and self.context_key in input:
                context = input[self.context_key]
                result_with_context = AIMessage(content=f"{context}\n{result.content}")
                history.add_messages([result_with_context])
            else:
                history.add_messages([result])
        
        return result

# Example usage
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're an assistant who's good at {ability}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

# Using a simple fake model for demonstration purposes
chain = prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
    context_key="context"
)

# First interaction
result = chain_with_history.invoke(
    {"ability": "math", "question": "What does cosine mean?", "context": "Classifying user input"},
    config={"configurable": {"session_id": "session_1"}}
)

print(result)

# Second interaction
result = chain_with_history.invoke(
    {"ability": "math", "question": "What's its inverse?", "context": "Classifying user input"},
    config={"configurable": {"session_id": "session_1"}}
)

print(result)

# Print the history to see the stored messages
history = get_by_session_id("session_1")
for message in history.messages:
    print(f"{message.__class__.__name__}: {message.content}")


content='Cosine is a trigonometric function that represents the ratio of the adjacent side of a right triangle to the hypotenuse. In simpler terms, cosine is a mathematical function that relates the angle of a triangle to the lengths of its sides. It is often abbreviated as cos in mathematical equations.' response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 34, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-8750d07c-f4ce-44b0-bcb5-415fb00d26f6-0'
content='The inverse of the cosine function is called the arccosine function, often denoted as cos^(-1) or arc cos. This function takes a value between -1 and 1 and returns the angle in radians whose cosine is that value. In other words, the arccosine function "undoes" the cosine function, allowing you to find the angle given the cosine value.' response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 111, 'total_token

In [96]:
store["session_1"].messages

[HumanMessage(content='What does cosine mean?'),
 AIMessage(content='Classifying user input\nCosine is a trigonometric function that represents the ratio of the adjacent side of a right triangle to the hypotenuse. In simpler terms, cosine is a mathematical function that relates the angle of a triangle to the lengths of its sides. It is often abbreviated as cos in mathematical equations.'),
 HumanMessage(content="What's its inverse?"),
 AIMessage(content='Classifying user input\nThe inverse of the cosine function is called the arccosine function, often denoted as cos^(-1) or arc cos. This function takes a value between -1 and 1 and returns the angle in radians whose cosine is that value. In other words, the arccosine function "undoes" the cosine function, allowing you to find the angle given the cosine value.')]